# CHAPTER 22 Dynamic Attributes and Properties

Example 22-1. Sample records from osconfeed.json; some field contents abbreviated

In [1]:
import json
with open('osconfeed.json') as fp:
    feed = json.load(fp)

In [2]:
sorted(feed['Schedule'].keys())

['conferences', 'events', 'speakers', 'venues']

In [3]:
for key, value in sorted(feed['Schedule'].items()):
    print(f'{len(value):3} {key}')

  1 conferences
  1 events
  1 speakers
  1 venues


In [4]:
feed['Schedule']['speakers'][-1]['name']

'Robert Lefkowitz'

In [5]:
feed['Schedule']['speakers'][-1]['serial']

157509

In [8]:
feed['Schedule']['speakers'][0]['name']

'Robert Lefkowitz'

Example 22-3 is a demonstration of FrozenJSON, and the source code is shown in
Example 22-4.

In [2]:
from frozenjson import FrozenJSON
raw_feed = json.load(open('osconfeed.json'))
feed = FrozenJSON(raw_feed)

In [4]:
len(feed.Schedule.speakers)

1

In [15]:
feed.keys()

dict_keys(['Schedule'])

In [16]:
sorted(feed.Schedule.keys())

['conferences', 'events', 'speakers', 'venues']

The Invalid Attribute Name Problem

In [3]:
student = FrozenJSON({'name': 'Jim Bo', 'class': 1982})

In [4]:
student.class

SyntaxError: invalid syntax (144001523.py, line 1)

In [5]:
getattr(student, 'class')

KeyError: 'class'

In [6]:
student.class_

1982

In [7]:
student = FrozenJSON({'2b': 'Jim Bo', 'class': 1982})

In [11]:
student.2b

SyntaxError: invalid decimal literal (2058554487.py, line 1)

Example 22-19. bulkfood_v1.py: the simplest LineItem class

In [1]:
class LineItem:
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

Example 22-20. A negative weight results in a negative subtotal

In [3]:
raisins = LineItem('Golden raisins', 10, 6.95)

In [4]:
raisins.subtotal()

69.5

In [5]:
raisins.weight = -20

In [6]:
raisins.subtotal()

-139.0

How to fix this problem of having negative value in item selection?

In [13]:
class LineItem2:
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price
    
    @property
    def weight(self):
        return self.__weight
    
    @weight.setter
    def weight(self, value):
        if value > 0:
            self.__weight = value
        else:
            raise ValueError('value must be > 0')


In [14]:
uzum = LineItem2('gara uzum', 10, 6.5)

In [15]:
uzum.subtotal()

65.0

In [21]:
uzum.weight = -1.2

ValueError: value must be > 0

In [19]:
uzum.weight = 1.2

In [20]:
uzum.subtotal()

7.8

In [24]:
uzum.__weight = -10

In [25]:
uzum.subtotal()

7.8

In [27]:
uzum.weight = 1

In [28]:
uzum.subtotal()

6.5

In [32]:
uzum._LineItem2__weight = 2

In [33]:
uzum._LineItem2__weight

2

In [34]:
uzum.subtotal()

13.0

In [35]:
uzum._LineItem2__weight = -2

In [36]:
uzum.subtotal()

-13.0

How to fix this?

In [37]:
class NonNegative:
    def __init__(self, name):
        self.name = name

    def __get__(self, instance, owner):
        return instance.__dict__[self.name]

    def __set__(self, instance, value):
        if value > 0:
            instance.__dict__[self.name] = value
        else:
            raise ValueError(f'{self.name} must be > 0')

In [38]:
class LineItem3:
    weight = NonNegative('weight')
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

In [39]:
armud = LineItem3('goi Armud', 10, 6.5)

In [40]:
armud.subtotal()

65.0

In [43]:
armud._LineItem3__weight = 2

In [44]:
armud.subtotal()

65.0

In [45]:
armud.weight = 3

In [46]:
armud.subtotal()

19.5